In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_pickle("../data/raw/not_null.pkl")

In [3]:
df.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,68407277,3600.0,3600.0,3600.0,36,13.99,123.03,C,C4,leadman,...,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,N,Cash,N
1,68355089,24700.0,24700.0,24700.0,36,11.99,820.28,C,C1,Engineer,...,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,N,Cash,N
2,68341763,20000.0,20000.0,20000.0,60,10.78,432.66,B,B4,truck driver,...,50.0,0.0,0.0,218418.0,18696.0,6200.0,14877.0,N,Cash,N
4,68476807,10400.0,10400.0,10400.0,60,22.45,289.91,F,F1,Contract Specialist,...,60.0,0.0,0.0,439570.0,95768.0,20300.0,88097.0,N,Cash,N
5,68426831,11950.0,11950.0,11950.0,36,13.44,405.18,C,C3,Veterinary Tecnician,...,100.0,0.0,0.0,16900.0,12798.0,9400.0,4000.0,N,Cash,N


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1230577 entries, 0 to 2260697
Data columns (total 87 columns):
 #   Column                      Non-Null Count    Dtype   
---  ------                      --------------    -----   
 0   id                          1230577 non-null  object  
 1   loan_amnt                   1230577 non-null  float64 
 2   funded_amnt                 1230577 non-null  float64 
 3   funded_amnt_inv             1230577 non-null  float64 
 4   term                        1230577 non-null  int64   
 5   int_rate                    1230577 non-null  float64 
 6   installment                 1230577 non-null  float64 
 7   grade                       1230577 non-null  category
 8   sub_grade                   1230577 non-null  category
 9   emp_title                   1230577 non-null  object  
 10  emp_length                  1230577 non-null  category
 11  home_ownership              1230577 non-null  category
 12  annual_inc                  1230577 non-nu

In [3]:
X = df.drop("loan_status", axis = 1)

In [4]:
y = df[["loan_status"]] 

In [5]:
def check_and_append_dummy_to_df(df):
    for column in df.columns:        
        if df[column].dtype.name == "category":
            dummy = pd.get_dummies(df[column], prefix=column, prefix_sep="_")
            df = pd.concat([df,dummy], axis=1)
            df.drop(column, axis=1, inplace=True)
    return df

In [6]:
X = check_and_append_dummy_to_df(X)

In [7]:
final_df = pd.concat([X,y], axis=1)

In [8]:
#drop columns
drop_column_name = ["id", "title", "zip_code", "emp_title"]
X.drop(drop_column_name, axis=1, inplace=True)

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

/home/d/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [10]:
# scaler = MinMaxScaler()
scaler = StandardScaler()

In [11]:
transformed = scaler.fit_transform(X)

In [12]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score

In [13]:
y["loan_status"] = y["loan_status"].astype("int")

/home/d/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
two_d_y = y["loan_status"].values.reshape(-1,1)

In [15]:
drop_column_data = df[["id"]]

In [19]:
drop_column_data

,id
0,68407277
1,68355089
2,68341763
4,68476807
5,68426831
...,...
2260688,89905081
2260690,88948836
2260691,89996426
2260692,90006534


In [16]:
len(drop_column_data), len(transformed)

(1230577, 1230577)

In [17]:
drop_column_data.to_pickle("../data/prod/ids.pkl")

In [18]:
final_2d = np.concatenate([transformed, two_d_y], axis=1)

In [19]:
final_columns = [*X.columns.values, *y.columns.values]

In [20]:
final_df = pd.DataFrame(final_2d, columns=final_columns)

In [21]:
final_df.to_pickle("../data/prod/scaler_data_standard.pkl")